In [ ]:
'''
（1）用17年的上一个月，同一个月，下一个月加权得到18年的预测值；

（2）然后按“省份”和“车型”统计16，17年的均值，17/16得到一个趋势因子，18年的预测值乘以这个因子能有飞跃的提升。
直接提交0.54，但是估计会被封，因为我提交过哈哈哈，最好融合一下再提交。

第一步可以提升，用17年的数据滑动平均或更复杂的加权得到18年预测值。
第二步中，可以考虑只按“省份”或只按“车型”统计均值相除得到因子，三个因子再加权一下得到更加鲁棒的趋势因子。
不仅是规则结果，模型预测的结果，也可以乘以这个因子得到提升；还可以考虑将这些因子放到模型里面去学，该是个强特。
'''

In [1]:
import pandas as pd
train = pd.read_csv('./data/train_sales_data.csv')
test = pd.read_csv('./data/evaluation_public.csv')
train16 = train[train['regYear'] == 2016][['adcode', 'model', 'regMonth', 'salesVolume']]
train17 = train[train['regYear'] == 2017][['adcode', 'model', 'regMonth', 'salesVolume']]
df16 = train16.groupby(['adcode', "model"], as_index=False)['salesVolume'].agg({"16mean": 'mean'}) # 按省份和车型统计均值
df17 = train17.groupby(['adcode', "model"], as_index=False)['salesVolume'].agg({"17mean": 'mean'}) # 按省份和车型统计均值
df = pd.merge(df17, df16, on=['adcode', 'model'], how='inner')
df['factor'] = df['17mean'] / df['16mean'] # 17年均值除以16年均值得到趋势因子
# 取出16年12月，17年1,2,3,4,5月，共6个月
df = pd.merge(df, train16[train16['regMonth'] == 12][['adcode', 'model', 'salesVolume']], on=['adcode', 'model'], how='left').rename(columns={'salesVolume': 0})
for m in range(1, 6):
    df = pd.merge(df, train17[train17['regMonth'] == m][['adcode', 'model', 'salesVolume']], on=['adcode', 'model'], how='left').rename(columns={'salesVolume': m})
result_df = pd.DataFrame()
temp_df = df[['adcode', 'model']].copy()
for m in range(1, 5):
 # 预测为上一年的上一个月，同一个月，下一个月的加权,再乘以趋势因子
    temp_df['forecastVolum'] = (df[m - 1].values * 0.25 + df[m].values * 0.5 + df[m + 1].values * 0.25) * df['factor']
    temp_df['regMonth'] = m
    result_df = result_df.append(temp_df, ignore_index=True, sort=False)
test = pd.merge(test[['id', 'adcode', 'model', 'regMonth']], result_df, how='left', on=['adcode', 'model', 'regMonth'])
test.loc[test['forecastVolum'] < 0, ['forecastVolum']] = 0
test[['id', 'forecastVolum']].round(0).astype(int).to_csv('submit/guize_new.csv', encoding='utf8', index=False)